# Merging gene expression data
In this R script we will merge the gene expression data files that were obtained from GEO and TCGA and pre-processed. Notebooks of the pre-processing of the data can be found at [GitHub](https://github.com/macsbio/inflammation_networks/tree/master/Jupyter-DataPreProcessing). In another notebook, which can be found in the repository, we cleaned the gene expression datasets. 

## The following step only works in RStudio. If working in another environment, please set the working directory properly and check if the working directory is correct. 

In [ ]:
# set wd to where script file is saved
setwd(dirname(rstudioapi::callFun("getActiveDocumentContext")$path))

In [1]:
# check wd
getwd()

[1] "C:/Users/Laurent/Jupyter - Cleaning & merging data"

In [2]:
# load libraries
library(dplyr)
library(biomaRt)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [3]:
# load data files
data1 <- read.table(file.path(getwd(),"Datasets", "Clean", "breast-cancer_stats_clean.txt"), header = T, sep = "\t")
data2 <- read.table(file.path(getwd(),"Datasets", "Clean", "lung-cancer-stats_clean.txt"), header = T, sep = "\t")
data3 <- read.table(file.path(getwd(),"Datasets", "Clean", "MUO-Lean_stats_clean.txt"), header = T, sep = "\t")
data4 <- read.table(file.path(getwd(),"Datasets", "Clean", "RA-control_GSE55235_stats_clean.txt"), header = T, sep = "\t")

data5 <- read.table(file.path(getwd(),"Datasets", "Clean", "RETT-control_FC_stats_clean.txt"), header = T, sep = "\t")
data6 <- read.table(file.path(getwd(),"Datasets", "Clean", "RETT-control_TC_stats_clean.txt"), header = T, sep = "\t")
data7 <- read.table(file.path(getwd(),"Datasets", "Clean", "SLE_stats_clean.txt"), header = T, sep = "\t")

head(data1)

ensembl_gene_id,logFC_BC,PValue_BC,entrezgene
ENSG00000000003,-0.19677472,0.733608386,7105
ENSG00000000005,-8.22242726,0.000011400,64102
ENSG00000000419,0.06848394,0.833040252,8813
ENSG00000000457,0.88768039,0.055529182,57147
ENSG00000000460,1.27175890,0.019866978,55732
ENSG00000000938,-3.09305336,0.000177035,2268


Now we have read in all cleaned data files, we would like to create a data frame that contains all unique entrezgene IDs of all these data files togethet. So if we merge all data files, we don't lose any data. We create a new dataframe with all unique entrezgene IDs, because we would like to have a data frame we add a column called "Nonsense". This columns contains the value 1 for every row. That way we have two dimensions, so we have a data frame. 

In [11]:
# get all unique entrezgene IDs from all datasets
dataStacked <- as.data.frame(cbind(data1$entrezgene,data2$entrezgene,data3$entrezgene,data4$entrezgene,
                     data5$entrezgene,data6$entrezgene,data7$entrezgene))
dataStacked <- stack(dataStacked)
dataStacked$Nonsense <- 1
dataStacked <- dataStacked[, -2]
dataStacked <- unique(dataStacked)
dataStacked <- na.omit(dataStacked)
colnames(dataStacked)[1] <- "entrezgene"

head(dataStacked)

Warning message in cbind(data1$entrezgene, data2$entrezgene, data3$entrezgene, data4$entrezgene, :
"number of rows of result is not a multiple of vector length (arg 1)"

entrezgene,Nonsense
7105,1
64102,1
8813,1
57147,1
55732,1
2268,1


In [12]:
# merge tables based on dataStacked, with all entrezgene IDs from all datasets
dataTotal <- dataStacked %>% left_join(data1, by = "entrezgene")
dataTotal <- dataTotal[,c(-2,-3)]
dataTotal <- dataTotal %>% left_join(data2, by = "entrezgene")
dataTotal <- dataTotal[,c(-4,-5)]
dataTotal <- dataTotal %>% left_join(data3, by = "entrezgene")
dataTotal <- dataTotal[,-6]
dataTotal <- dataTotal %>% left_join(data4, by = "entrezgene")
dataTotal <- dataTotal[,-8]

dataTotal <- dataTotal %>% left_join(data5, by = "entrezgene")
dataTotal <- dataTotal[,c(-10,-11)]
dataTotal <- dataTotal %>% left_join(data6, by = "entrezgene")
dataTotal <- dataTotal[,c(-12,-13)]
dataTotal <- dataTotal %>% left_join(data7, by = "entrezgene")

head(dataTotal)

entrezgene,logFC_BC,PValue_BC,logFC_LC,PValue_LC,logFC_MUO,PValue_MUO,logFC_RA235,PValue_RA235,logFC_RETT_FC,PValue_RETT_FC,logFC_RETT_TC,PValue_RETT_TC,logFC_SLE,PValue_SLE
7105,-0.19677472,0.733608386,1.2109203,0.030285815,-0.19539969,0.0754891770,-0.30046001,1.865526e-02,0.53858538,0.00140977,0.11512825,0.4126980,-0.4260808,0.0003047616
64102,-8.22242726,0.000011400,1.6015344,0.171398244,1.80412948,0.0000016200,-0.04860816,8.196471e-01,0.08571963,0.30895631,0.05364937,0.5195787,NA,NA
8813,0.06848394,0.833040252,0.4674649,0.294138000,-0.40356445,0.0001569100,-0.34473391,2.328536e-02,NA,NA,NA,NA,0.0000000,1.0000000000
57147,0.88768039,0.055529182,0.0544462,0.891635248,-0.15488979,0.1700401430,0.08311677,4.803042e-01,-0.10965825,0.24317441,-0.07608135,0.4200971,0.0000000,1.0000000000
55732,1.27175890,0.019866978,1.4376887,0.004173094,0.08653913,0.0135634820,-0.21473569,3.695494e-02,0.01494288,0.91233348,-0.14298550,0.3013756,0.0000000,1.0000000000
2268,-3.09305336,0.000177035,-2.9959429,0.000004940,0.55657959,0.0000000911,1.34143362,6.749178e-06,-0.24507187,0.33607730,-0.16621479,0.5103585,-0.2494521,0.4988292847


We have merged the data files together! Now we only have to add hgnc symbols where ever it is possible and save the file!

In [13]:
# add hgnc symbol to dataTotal
ensembl <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")
mygenes <- dataTotal

my.genes1 <- getBM(
  attributes = c('entrezgene', 'hgnc_symbol'), 
  filters = 'entrezgene',
  values = mygenes,
  mart = ensembl
)

# clean merged data file
dataTotal <- dataTotal %>% left_join(my.genes1, by = "entrezgene")
dataTotal <- dataTotal[,c(1,16,2,3,4,5,6,7,8,9,10,11,12,13,14,15)]
dataTotal <- unique(dataTotal)

head(dataTotal)

entrezgene,hgnc_symbol,logFC_BC,PValue_BC,logFC_LC,PValue_LC,logFC_MUO,PValue_MUO,logFC_RA235,PValue_RA235,logFC_RETT_FC,PValue_RETT_FC,logFC_RETT_TC,PValue_RETT_TC,logFC_SLE,PValue_SLE
7105,TSPAN6,-0.19677472,0.733608386,1.2109203,0.030285815,-0.19539969,0.0754891770,-0.30046001,1.865526e-02,0.53858538,0.00140977,0.11512825,0.4126980,-0.4260808,0.0003047616
64102,TNMD,-8.22242726,0.000011400,1.6015344,0.171398244,1.80412948,0.0000016200,-0.04860816,8.196471e-01,0.08571963,0.30895631,0.05364937,0.5195787,NA,NA
8813,DPM1,0.06848394,0.833040252,0.4674649,0.294138000,-0.40356445,0.0001569100,-0.34473391,2.328536e-02,NA,NA,NA,NA,0.0000000,1.0000000000
57147,SCYL3,0.88768039,0.055529182,0.0544462,0.891635248,-0.15488979,0.1700401430,0.08311677,4.803042e-01,-0.10965825,0.24317441,-0.07608135,0.4200971,0.0000000,1.0000000000
55732,C1orf112,1.27175890,0.019866978,1.4376887,0.004173094,0.08653913,0.0135634820,-0.21473569,3.695494e-02,0.01494288,0.91233348,-0.14298550,0.3013756,0.0000000,1.0000000000
2268,FGR,-3.09305336,0.000177035,-2.9959429,0.000004940,0.55657959,0.0000000911,1.34143362,6.749178e-06,-0.24507187,0.33607730,-0.16621479,0.5103585,-0.2494521,0.4988292847


In [14]:
# save file
write.table(dataTotal, file.path(getwd(), "DataSets", "merged_data_final.txt"), row.names = F, col.names = T, sep = "\t", quote = F)